In [1]:
import pandas as pd
import numpy as np

Files were provided by Bin Sun 

In [11]:
#file = "data/pH3_populations.dat"
file = "data/pH7_populations.dat"

Load and label

In [12]:
df =pd.read_csv(file,sep="\s+",skiprows=7,header=None)

In [13]:
df  = df.drop([0,2,4,6,8,10,12],axis="columns")
df = df.replace(np.nan, 0)

Need to verify charge/protonation states 
<code>
GL -1, 0, 1, 2.  ???
AS -1, 0, 1, 2.  ???
CS 0, 1
</code>



In [14]:
lookUp ={
    'GL4': np.array([-1,0,1,2,3]),
    'AS4': np.array([-1,0,1,2,3]),
    'CYS': np.array([ 0,1,0,0,0])
}
v = df.iloc[0]


In [15]:
df.rename(columns={
    1: 'res', 
    3: 'state0',
    5: 'state1',
    7: 'state2',    
    9: 'state3',    
    11: 'state4',    
}, inplace=True)


Add probabilities and charge

In [16]:
def CalcAvgCharge(df,ar,res='GL4'):
    dfss = df[ df['res'] == res ]

    for index, row in dfss.iterrows():
        values = row[['state0','state1','state2','state3','state4']].values  # different states 
        dotp = np.dot(values,lookUp[res])
        #df[index]['charge']=dotp
        ar[int(index)]=dotp
 
    #print(ar)
#df[ df[1] == "GL4" ]['charge'] =0   

In [17]:
charges = np.zeros( len( df.index ) )    
CalcAvgCharge(df,charges,res='GL4')
CalcAvgCharge(df,charges,res='AS4')
CalcAvgCharge(df,charges,res='CYS')

df['charge'] = charges

double check, since it might only be the total number, not total charge

In [18]:
tot = len( df.index )
print("WARNING: only for charged residues, not entire prot")
dfss = df[ df['charge'] < 0 ]
rhoNeg = dfss['charge'].sum() / tot

dfss = df[ df['charge'] > 0 ]
rhoPos = dfss['charge'].sum() / tot


In [19]:
print(rhoNeg,rhoPos)

-0.621903448275862 6.89655172413793e-08
